# JSON Packet formation from MQTT response

In [1]:
import paho.mqtt.client as mqtt
import json

resp ={}
gateway_info = {}
gatewayid = []
macidsetlist = []
macidlist = []
rssilist = []
pckt = {}

# The callback for when the client receives a CONNECT response from the server.
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully")
    else:
        print("Connect returned result code: " + str(rc))

#The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
#     print("Received message: " + msg.topic + " -> " + msg.payload.decode("utf-8"))
    resp = json.loads(msg.payload.decode("utf-8"))
    formatting(resp)
    
def formatting(gateway1_resp):
    macidset = set() # list of sets containing beacon ids from each gateway response used to form a superset of unique beacon ids 
    macid = []       # List of lists containing beacon ids from each gateway response used to find the idices to access respective RSSI and Gateway IDs
    rssi = []        # List of lists containing beacon RSSI used to find dist fom rssi_dist dictionary
    gatewayid.append(gateway1_resp["id"])
#     print("Gatewayid list {0}".format(gatewayid))
    beacon_data = gateway1_resp["raw_beacons_data"]
    beaconlist = beacon_data.split(";")
    # RSSI to Distance (in cm) conversion dictinary
    rssi_dist = {'E4':0,'E3':0,'E2':0,'E1':0,'E0':0,'DF':0,'DE':0,'DD':0,'DC':3,'DB':6,'DA':9,'D9':12,'D8':15,'D7':18,'D6':21,'D5':24,'D4':27,'D3':30,'D2':33,'D1':36,'D0':39,'CF':42,'CE':45,'CD':48,'CC':51,'CB':54,'CA':60,'C9':66,'C8':72,'C7':78,'C6':84,'C5':90,'C4':96,'C3':102,'C2':108,'C1':114,'C0':120,'BF':130,'BE':140,'BD':150,'BC':170,'BB':190,'BA':210,'B9':230,'B8':250,'B7':270,'B6':290,'B5':310,'B4':350,'B3':390,'B2':430,'B1':470,'B0':510,'AF':550,'AE':590,'AD':630,'AC':670,'AB':710,'AA':750,'A9':790,'A8':830,'A7':870,'A6':910,'A5':950,'A4':990,'A3':1030,'A2':1070,'A1':1110,'A0':1150}
    for i in range(0,len(beaconlist)-1):
        payload = beaconlist[i]
        macidset.add(payload[:12])
        macid.append(payload[:12])
        rssi.append(payload[-2:])
    macidsetlist.append(macidset)
    macidlist.append(macid)
    rssilist.append(rssi)
    if(len(gatewayid) == 2): # Since we have only 2 gatewways for now
        print("Gateway ID {}".format(gatewayid))
        print("MACID list {0}".format(macidlist))
        print("RSSI list {0}".format(rssilist))
        beacon_superset = list((set().union(*macidlist))) # Set union operation to find unique beacon ids from all the gateways
        print("Beacon Sample space {0}".format(beacon_superset))
        print("__________________________________________________")
        for beaconnmbr in range(0,len(beacon_superset)):
            pckt["beacon"+str(beaconnmbr+1)] = {}
            pckt["beacon"+str(beaconnmbr+1)]["beacon_id"] = beacon_superset[beaconnmbr]
            pckt["beacon"+str(beaconnmbr+1)]["scanned_by"] = {}
            for gatewaynmbr in range(0,len(gatewayid)): # Since the number of beacon macid responses is equal to the number of gateways
#                 print("gatewaynmbr: {0}".format(gatewaynmbr))
                pckt["beacon"+str(beaconnmbr+1)]["scanned_by"]["gateway"+str(gatewaynmbr+1)] = {}
                pckt["beacon"+str(beaconnmbr+1)]["scanned_by"]["gateway"+str(gatewaynmbr+1)]["gateway_id"] = gatewayid[gatewaynmbr]
                for beacons in range(0,len(macidlist[gatewaynmbr])):
#                     print("COMBOS: {0} , {1}".format(beacon_superset[beaconnmbr],macidlist[gatewaynmbr][beacons]))
                    if(beacon_superset[beaconnmbr] == macidlist[gatewaynmbr][beacons]):
#                         print("Common: {0} == {1}".format(beacon_superset[beaconnmbr],macidlist[gatewaynmbr][beacons]))
#                         print("Before gatewaynmbr : {0}, beacons: {1}".format(gatewaynmbr,beacons))
                        pckt["beacon"+str(beaconnmbr+1)]["scanned_by"]["gateway"+str(gatewaynmbr+1)]["rssi"] = rssilist[gatewaynmbr][beacons]
                        pckt["beacon"+str(beaconnmbr+1)]["scanned_by"]["gateway"+str(gatewaynmbr+1)]["dist"] = rssi_dist[rssilist[gatewaynmbr][beacons]]
                        print("Beacon{0} scanned by gateway{1} {2} with RSSI {3} and placed {4} cm away".format(beaconnmbr+1,gatewaynmbr+1,gatewayid[gatewaynmbr],rssilist[gatewaynmbr][beacons],rssi_dist[rssilist[gatewaynmbr][beacons]]))
                        break
                    else:
                        pckt["beacon"+str(beaconnmbr+1)]["scanned_by"]["gateway"+str(gatewaynmbr+1)]["rssi"] = "nan"
                        pckt["beacon"+str(beaconnmbr+1)]["scanned_by"]["gateway"+str(gatewaynmbr+1)]["dist"] = "nan"
#                         print("Beacon{0} not scanned by gateway{1} {2} as the RSSI is {3}".format(beaconnmbr+1,gatewaynmbr+1,gatewayid[gatewaynmbr],"nan"))
#                     print("Packet {0}".format(pckt))
        print("__________________________________________________")
        print("Ble Packet {0}".format(pckt))
        gatewayid.clear()
        macidlist.clear()
        rssilist.clear()
        macidsetlist.clear()
        beacon_superset.clear()
        pckt.clear()
        print("*********************************************************************************")

# create the client
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# connect to HiveMQ Cloud on port 8883
client.connect("broker.hivemq.com", 1883)

# subscribe to the topic "my/test/topic"
client.subscribe([("/ufobeacons",0),("/ufobeacons1",0)])


# Blocking call that processes network traffic, dispatches callbacks and handles reconnecting.
client.loop_forever()

Connected successfully
Gateway ID ['F4844C4D4845', '28EC9A73BB1B']
MACID list [['55464F604C56', '55464FF34227', '55464F601993'], ['55464F604C56']]
RSSI list [['CE', 'CE', 'BB'], ['B7']]
Beacon Sample space ['55464F601993', '55464FF34227', '55464F604C56']
__________________________________________________
Beacon1 scanned by gateway1 F4844C4D4845 with RSSI BB and placed 190 cm away
Beacon2 scanned by gateway1 F4844C4D4845 with RSSI CE and placed 45 cm away
Beacon3 scanned by gateway1 F4844C4D4845 with RSSI CE and placed 45 cm away
Beacon3 scanned by gateway2 28EC9A73BB1B with RSSI B7 and placed 270 cm away
__________________________________________________
Ble Packet {'beacon1': {'beacon_id': '55464F601993', 'scanned_by': {'gateway1': {'gateway_id': 'F4844C4D4845', 'rssi': 'BB', 'dist': 190}, 'gateway2': {'gateway_id': '28EC9A73BB1B', 'rssi': 'nan', 'dist': 'nan'}}}, 'beacon2': {'beacon_id': '55464FF34227', 'scanned_by': {'gateway1': {'gateway_id': 'F4844C4D4845', 'rssi': 'CE', 'dist': 4

Gateway ID ['F4844C4D4845', '28EC9A73BB1B']
MACID list [['55464F61639D', '55464F604C56', '55464FF34227'], ['55464F604C56', '55464FF34227']]
RSSI list [['A9', 'C8', 'D2'], ['B8', 'B6']]
Beacon Sample space ['55464F604C56', '55464FF34227', '55464F61639D']
__________________________________________________
Beacon1 scanned by gateway1 F4844C4D4845 with RSSI C8 and placed 72 cm away
Beacon1 scanned by gateway2 28EC9A73BB1B with RSSI B8 and placed 250 cm away
Beacon2 scanned by gateway1 F4844C4D4845 with RSSI D2 and placed 33 cm away
Beacon2 scanned by gateway2 28EC9A73BB1B with RSSI B6 and placed 290 cm away
Beacon3 scanned by gateway1 F4844C4D4845 with RSSI A9 and placed 790 cm away
__________________________________________________
Ble Packet {'beacon1': {'beacon_id': '55464F604C56', 'scanned_by': {'gateway1': {'gateway_id': 'F4844C4D4845', 'rssi': 'C8', 'dist': 72}, 'gateway2': {'gateway_id': '28EC9A73BB1B', 'rssi': 'B8', 'dist': 250}}}, 'beacon2': {'beacon_id': '55464FF34227', 'scanned_

Gateway ID ['F4844C4D4845', '28EC9A73BB1B']
MACID list [['55464F61639D', '55464F604C56', '55464FF34227'], ['55464F604C56', '55464FF34227']]
RSSI list [['AB', 'C7', 'D2'], ['B1', 'B7']]
Beacon Sample space ['55464F604C56', '55464FF34227', '55464F61639D']
__________________________________________________
Beacon1 scanned by gateway1 F4844C4D4845 with RSSI C7 and placed 78 cm away
Beacon1 scanned by gateway2 28EC9A73BB1B with RSSI B1 and placed 470 cm away
Beacon2 scanned by gateway1 F4844C4D4845 with RSSI D2 and placed 33 cm away
Beacon2 scanned by gateway2 28EC9A73BB1B with RSSI B7 and placed 270 cm away
Beacon3 scanned by gateway1 F4844C4D4845 with RSSI AB and placed 710 cm away
__________________________________________________
Ble Packet {'beacon1': {'beacon_id': '55464F604C56', 'scanned_by': {'gateway1': {'gateway_id': 'F4844C4D4845', 'rssi': 'C7', 'dist': 78}, 'gateway2': {'gateway_id': '28EC9A73BB1B', 'rssi': 'B1', 'dist': 470}}}, 'beacon2': {'beacon_id': '55464FF34227', 'scanned_

Gateway ID ['F4844C4D4845', '28EC9A73BB1B']
MACID list [['55464F61639D', '55464FF34227', '55464F601993'], ['55464F604C56', '55464FF34227']]
RSSI list [['A9', 'D0', 'B2'], ['B8', 'B1']]
Beacon Sample space ['55464F604C56', '55464F601993', '55464FF34227', '55464F61639D']
__________________________________________________
Beacon1 scanned by gateway2 28EC9A73BB1B with RSSI B8 and placed 250 cm away
Beacon2 scanned by gateway1 F4844C4D4845 with RSSI B2 and placed 430 cm away
Beacon3 scanned by gateway1 F4844C4D4845 with RSSI D0 and placed 39 cm away
Beacon3 scanned by gateway2 28EC9A73BB1B with RSSI B1 and placed 470 cm away
Beacon4 scanned by gateway1 F4844C4D4845 with RSSI A9 and placed 790 cm away
__________________________________________________
Ble Packet {'beacon1': {'beacon_id': '55464F604C56', 'scanned_by': {'gateway1': {'gateway_id': 'F4844C4D4845', 'rssi': 'nan', 'dist': 'nan'}, 'gateway2': {'gateway_id': '28EC9A73BB1B', 'rssi': 'B8', 'dist': 250}}}, 'beacon2': {'beacon_id': '554

Gateway ID ['F4844C4D4845', '28EC9A73BB1B']
MACID list [['55464F604C56', '55464FF34227', '55464F601993'], ['55464F604C56', '55464FF34227']]
RSSI list [['CF', 'CE', 'BB'], ['B2', 'B4']]
Beacon Sample space ['55464F601993', '55464FF34227', '55464F604C56']
__________________________________________________
Beacon1 scanned by gateway1 F4844C4D4845 with RSSI BB and placed 190 cm away
Beacon2 scanned by gateway1 F4844C4D4845 with RSSI CE and placed 45 cm away
Beacon2 scanned by gateway2 28EC9A73BB1B with RSSI B4 and placed 350 cm away
Beacon3 scanned by gateway1 F4844C4D4845 with RSSI CF and placed 42 cm away
Beacon3 scanned by gateway2 28EC9A73BB1B with RSSI B2 and placed 430 cm away
__________________________________________________
Ble Packet {'beacon1': {'beacon_id': '55464F601993', 'scanned_by': {'gateway1': {'gateway_id': 'F4844C4D4845', 'rssi': 'BB', 'dist': 190}, 'gateway2': {'gateway_id': '28EC9A73BB1B', 'rssi': 'nan', 'dist': 'nan'}}}, 'beacon2': {'beacon_id': '55464FF34227', 'scan

Gateway ID ['F4844C4D4845', '28EC9A73BB1B']
MACID list [['55464F601993', '55464F604C56', '55464FF34227'], ['55464F604C56', '55464FF34227']]
RSSI list [['BB', 'CF', 'CE'], ['B8', 'B2']]
Beacon Sample space ['55464F601993', '55464FF34227', '55464F604C56']
__________________________________________________
Beacon1 scanned by gateway1 F4844C4D4845 with RSSI BB and placed 190 cm away
Beacon2 scanned by gateway1 F4844C4D4845 with RSSI CE and placed 45 cm away
Beacon2 scanned by gateway2 28EC9A73BB1B with RSSI B2 and placed 430 cm away
Beacon3 scanned by gateway1 F4844C4D4845 with RSSI CF and placed 42 cm away
Beacon3 scanned by gateway2 28EC9A73BB1B with RSSI B8 and placed 250 cm away
__________________________________________________
Ble Packet {'beacon1': {'beacon_id': '55464F601993', 'scanned_by': {'gateway1': {'gateway_id': 'F4844C4D4845', 'rssi': 'BB', 'dist': 190}, 'gateway2': {'gateway_id': '28EC9A73BB1B', 'rssi': 'nan', 'dist': 'nan'}}}, 'beacon2': {'beacon_id': '55464FF34227', 'scan

KeyboardInterrupt: 